In [28]:
import time


import polars as pl

In [29]:
IN = open("d2_in.txt")
with IN as f:
    ranges = f.readline().rstrip("\r\n").split(",")
    ranges = [r.split("-") for r in ranges]
    ranges_lst = [(int(r[0]), int(r[1])) for r in ranges]
    ranges = [range(r[0], r[1]+1) for r in ranges_lst]

In [ ]:
df = pl.DataFrame({"i": ranges[0]})
for r in ranges[1:]:
    df = pl.concat([df, pl.DataFrame({"i": r})])

Part 1

In [31]:
t0 = time.time()
print(
    df.with_columns(l=pl.col("i").cast(pl.String).str.len_chars().cast(pl.Int8) // 2)
    .with_columns(
        a=pl.col("i") // (10**pl.col("l")),
        b=pl.col("i") % (10**pl.col("l")),
    )
    .filter(pl.col("a") == pl.col("b"))
    .select(pl.col("i").sum())
)
dt = time.time() - t0
print(f"Time: {dt:.2f}")

shape: (1, 1)
┌─────────────┐
│ i           │
│ ---         │
│ i64         │
╞═════════════╡
│ 21898734247 │
└─────────────┘
Time: 0.06


Part 2

In [25]:
t0 = time.time()
print(
    df.with_columns(l=pl.col("i").cast(pl.String).str.len_chars().cast(pl.Int8))
    .with_columns(lp=pl.int_ranges(pl.col("l") // 2, pl.col("l")).cast(pl.List(pl.Int8)))
    .explode([pl.col("lp")])
    .with_columns(
        j=
        pl.col("i")//(pl.lit(10).cast(pl.Int64)**pl.col("lp")) +
        (pl.col("i")%(pl.lit(10).cast(pl.Int64)**pl.col("lp"))) * 
        (pl.lit(10).cast(pl.Int64)**(pl.col("l")-pl.col("lp")))
    )
    .filter(pl.col("i") == pl.col("j"), pl.col("l") > 1)
    .select(pl.col("i").unique().sum())
)
dt = time.time() - t0
print(f"Time: {dt:.2f}")

shape: (1, 1)
┌─────────────┐
│ i           │
│ ---         │
│ i64         │
╞═════════════╡
│ 28915664389 │
└─────────────┘
Time: 0.47
